In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')
import plotutils

In [3]:
import fitbit
import os
import getpass
import numpy as np
import datetime

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [22]:
os.environ['client_id'] = getpass.getpass('Enter Client ID:')
os.environ['client_secret'] = getpass.getpass('Enter Client Secret:')

Enter Client ID:········
Enter Client Secret:········


In [23]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[27/Oct/2019:19:06:03] ENGINE Listening for SIGTERM.
[27/Oct/2019:19:06:03] ENGINE Listening for SIGHUP.
[27/Oct/2019:19:06:03] ENGINE Listening for SIGUSR1.
[27/Oct/2019:19:06:03] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[27/Oct/2019:19:06:03] ENGINE Started monitor thread 'Autoreloader'.
[27/Oct/2019:19:06:03] ENGINE Serving on http://127.0.0.1:8080
[27/Oct/2019:19:06:03] ENGINE Bus STARTED
127.0.0.1 - - [27/Oct/2019:19:06:05] "GET /?code=7b29a2ba1b4b8e04d09a682cbc1ee45f61f86bf4&state=xoqdZ7X6WyNzLs7hMrkxwTamtk0Ukf HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
[27/Oct/2019:19:06:06] ENGINE Bus STOPPING
[27/Oct/2019:19:06:06] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[27/Oct/2019:19:06:06] ENGINE Stopped thread 'Autoreloader'.
[27/Oct/2019:19:06:06] ENGINE Bus STOPPED
[27/Oct/2019:19:06:06] ENGI

In [24]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()

In [25]:
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

In [26]:
sleep = auth_client.sleep()['summary']['stages']

In [27]:
plotutils.sleep_summary_plot(sleep)